In [225]:
import numpy as np
import pandas as pd
import h5py
import scipy.sparse as sps
from datetime import timedelta
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)


In [226]:
filename = '../data/all_hourly_data.h5'

In [227]:
with h5py.File(filename, "r") as f:
    print("Keys: %s" % f.keys())


Keys: <KeysViewHDF5 ['codes', 'interventions', 'patients', 'vitals_labs', 'vitals_labs_mean']>


In [228]:
patients = pd.read_hdf(filename, key="patients")

one_hot = pd.get_dummies(patients[["gender", "admission_type", "first_careunit"]])
patients = patients.join(one_hot)
#display(patients.columns)

# include only patients with 6h < los < 600h or same for los_icu, but los_icu is in days
patients["los"] = (patients["dischtime"] - patients["admittime"])
patients = patients[((patients["los"] > timedelta(hours=6)) & (patients["los"] < timedelta(hours=600))) | ((patients["los_icu"]*24 > 6) & (patients["los_icu"]*24 < 600))]

patients = patients[patients["age"] > 18]

# ignore following dummies, common practice to exclude 1 dummy per category
# gender_M
# admission_type_ELECTIVE
# first_careunit_CCU

patients = patients[["age", "gender_F", "admission_type_EMERGENCY", "admission_type_URGENT", "first_careunit_CSRU", "first_careunit_MICU", "first_careunit_SICU", "first_careunit_TSICU"]]
print(patients.shape)
display(patients.head())


(37543, 8)


,,,age,gender_F,admission_type_EMERGENCY,admission_type_URGENT,first_careunit_CSRU,first_careunit_MICU,first_careunit_SICU,first_careunit_TSICU
subject_id,hadm_id,icustay_id,,,,,,,,
3,145834,211552,76.53,0,1,0,0,1,0,0
4,185777,294638,47.85,1,1,0,0,1,0,0
6,107064,228232,65.94,1,0,0,0,0,1,0
9,150750,220597,41.79,0,1,0,0,1,0,0
11,194540,229441,50.15,1,1,0,0,0,1,0


In [229]:
interventions = pd.read_hdf(filename, key="interventions")
interventions["vasopressor"] = interventions[['vaso', 'vasopressin']].any(axis='columns').astype(int)
interventions = interventions[["vasopressor"]]
print(interventions.shape)
display(interventions.head())

(3183638, 1)


vasopressor
subject_id hadm_id icustay_id hours_in             
3          145834  211552     0                   0
                              1                   1
                              2                   1
                              3                   1
                              4                   1

In [230]:
# exclude patients that get vasopressor in the first 6h
excluded_subject_ids = interventions[interventions.index.get_level_values("hours_in").isin(range(6)) & interventions["vasopressor"] == 1]
excluded_subject_ids = excluded_subject_ids.index.get_level_values("subject_id").unique()
excluded_subject_ids.shape

(8406,)

In [231]:
outcome = pd.DataFrame(interventions.groupby(["subject_id", "hadm_id", "icustay_id"])["vasopressor"].agg("max"))
print(outcome.shape)
display(outcome.head())


(37543, 1)


,,,vasopressor
subject_id,hadm_id,icustay_id,
3,145834,211552,1
4,185777,294638,0
6,107064,228232,0
9,150750,220597,1
11,194540,229441,0


In [232]:
vitals_labs_mean = pd.read_hdf(filename, key="vitals_labs_mean")

# Paper name -> df name
# map (mean arterial pressure) -> pulmonary artery pressure mean
# spontaneousrr (spontaneous respiratory rate) -> respiratory rate
# urine (urine output) -> not found! :/
# alt (alanine transaminase) -> alanine aminotransferase
# ast (aspartate aminotransferase) -> asparate aminotransferase
# inr (international normalised ratio) -> prothrombin time inr
mask = vitals_labs_mean.columns.get_level_values("LEVEL2").isin(["diastolic blood pressure", "fraction inspired oxygen", "glascow coma scale total", "heart rate", 
                                                                 "pulmonary artery pressure mean", "systolic blood pressure", "respiratory rate", "oxygen saturation",
                                                                 "temperature", "urine output", "blood urea nitrogen", "magnesium", "platelets", "sodium", "alanine aminotransferase",
                                                                 "hematocrit", "partial pressure of oxygen", "asparate aminotransferase", "potassium", "white blood cell count",
                                                                 "bicarbonate", "creatinine", "lactate", "partial pressure of carbon dioxide", "glucose", "prothrombin time inr",
                                                                 "hemoglobin", "bilirubin"])

#display(vitals_labs_mean.columns[vitals_labs_mean.columns.get_level_values("LEVEL2").str.contains("rate")])
vitals_labs_mean = vitals_labs_mean.iloc[:, mask]
print(vitals_labs_mean.shape)
display(vitals_labs_mean.head())


(3183638, 27)


LEVEL2                                 alanine aminotransferase  \
Aggregation Function                                       mean   
subject_id hadm_id icustay_id hours_in                            
3          145834  211552     0                           25.00   
                              1                             NaN   
                              2                             NaN   
                              3                             NaN   
                              4                             NaN   

LEVEL2                                 asparate aminotransferase bicarbonate  \
Aggregation Function                                        mean        mean   
subject_id hadm_id icustay_id hours_in                                         
3          145834  211552     0                            69.00       16.33   
                              1                              NaN         NaN   
                              2                              NaN         NaN   
                              3                              NaN         NaN   
                              4                              NaN         NaN   

LEVEL2                                 bilirubin blood urea nitrogen  \
Aggregation Function                        mean                mean   
subject_id hadm_id icustay_id hours_in                                 
3          145834  211552     0             0.80               44.20   
                              1              NaN                 NaN   
                              2              NaN                 NaN   
                              3              NaN                 NaN   
                              4              NaN                 NaN   

LEVEL2                                 creatinine diastolic blood pressure  \
Aggregation Function                         mean                     mean   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0              2.60                    39.67   
                              1               NaN                    44.12   
                              2               NaN                    47.33   
                              3               NaN                    64.50   
                              4               NaN                    63.00   

LEVEL2                                 fraction inspired oxygen  \
Aggregation Function                                       mean   
subject_id hadm_id icustay_id hours_in                            
3          145834  211552     0                             NaN   
                              1                             NaN   
                              2                             NaN   
                              3                             NaN   
                              4                             NaN   

LEVEL2                                 glascow coma scale total glucose  \
Aggregation Function                                       mean    mean   
subject_id hadm_id icustay_id hours_in                                    
3          145834  211552     0                             NaN  198.56   
                              1                             NaN     NaN   
                              2                             NaN  267.00   
                              3                             NaN     NaN   
                              4                             NaN     NaN   

LEVEL2                                 heart rate hematocrit hemoglobin  \
Aggregation Function                         mean       mean       mean   
subject_id hadm_id icustay_id hours_in                                    
3          145834  211552     0            143.00      27.03       8.72   
                              1            153.75        NaN        NaN   
                              2            137.75      30.00      10.10   
                              3 

In [239]:
# expect (N x V x T)
T = 6
N = vitals_labs_mean.index.get_level_values("subject_id").unique().size
V = vitals_labs_mean.shape[1]

# use first 6 hours

vitals_labs_mean1 = vitals_labs_mean.to_numpy().reshape(N, V, T)

29137
27


In [233]:
indicators = pd.DataFrame(~vitals_labs_mean.isna())
print(indicators.shape)
display(indicators.head())


(3183638, 27)


LEVEL2                                 alanine aminotransferase  \
Aggregation Function                                       mean   
subject_id hadm_id icustay_id hours_in                            
3          145834  211552     0                            True   
                              1                           False   
                              2                           False   
                              3                           False   
                              4                           False   

LEVEL2                                 asparate aminotransferase bicarbonate  \
Aggregation Function                                        mean        mean   
subject_id hadm_id icustay_id hours_in                                         
3          145834  211552     0                             True        True   
                              1                            False       False   
                              2                            False       False   
                              3                            False       False   
                              4                            False       False   

LEVEL2                                 bilirubin blood urea nitrogen  \
Aggregation Function                        mean                mean   
subject_id hadm_id icustay_id hours_in                                 
3          145834  211552     0             True                True   
                              1            False               False   
                              2            False               False   
                              3            False               False   
                              4            False               False   

LEVEL2                                 creatinine diastolic blood pressure  \
Aggregation Function                         mean                     mean   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0              True                     True   
                              1             False                     True   
                              2             False                     True   
                              3             False                     True   
                              4             False                     True   

LEVEL2                                 fraction inspired oxygen  \
Aggregation Function                                       mean   
subject_id hadm_id icustay_id hours_in                            
3          145834  211552     0                           False   
                              1                           False   
                              2                           False   
                              3                           False   
                              4                           False   

LEVEL2                                 glascow coma scale total glucose  \
Aggregation Function                                       mean    mean   
subject_id hadm_id icustay_id hours_in                                    
3          145834  211552     0                           False    True   
                              1                           False   False   
                              2                           False    True   
                              3                           False   False   
                              4                           False   False   

LEVEL2                                 heart rate hematocrit hemoglobin  \
Aggregation Function                         mean       mean       mean   
subject_id hadm_id icustay_id hours_in                                    
3          145834  211552     0              True       True       True   
                              1              True      False      False   
                              2              True       True       True   
                              3 

In [234]:
vitals_labs_mean.fillna(0, inplace=True)
print(vitals_labs_mean.shape)
display(vitals_labs_mean.head())


(3183638, 27)


LEVEL2                                 alanine aminotransferase  \
Aggregation Function                                       mean   
subject_id hadm_id icustay_id hours_in                            
3          145834  211552     0                           25.00   
                              1                            0.00   
                              2                            0.00   
                              3                            0.00   
                              4                            0.00   

LEVEL2                                 asparate aminotransferase bicarbonate  \
Aggregation Function                                        mean        mean   
subject_id hadm_id icustay_id hours_in                                         
3          145834  211552     0                            69.00       16.33   
                              1                             0.00        0.00   
                              2                             0.00        0.00   
                              3                             0.00        0.00   
                              4                             0.00        0.00   

LEVEL2                                 bilirubin blood urea nitrogen  \
Aggregation Function                        mean                mean   
subject_id hadm_id icustay_id hours_in                                 
3          145834  211552     0             0.80               44.20   
                              1             0.00                0.00   
                              2             0.00                0.00   
                              3             0.00                0.00   
                              4             0.00                0.00   

LEVEL2                                 creatinine diastolic blood pressure  \
Aggregation Function                         mean                     mean   
subject_id hadm_id icustay_id hours_in                                       
3          145834  211552     0              2.60                    39.67   
                              1              0.00                    44.12   
                              2              0.00                    47.33   
                              3              0.00                    64.50   
                              4              0.00                    63.00   

LEVEL2                                 fraction inspired oxygen  \
Aggregation Function                                       mean   
subject_id hadm_id icustay_id hours_in                            
3          145834  211552     0                            0.00   
                              1                            0.00   
                              2                            0.00   
                              3                            0.00   
                              4                            0.00   

LEVEL2                                 glascow coma scale total glucose  \
Aggregation Function                                       mean    mean   
subject_id hadm_id icustay_id hours_in                                    
3          145834  211552     0                            0.00  198.56   
                              1                            0.00    0.00   
                              2                            0.00  267.00   
                              3                            0.00    0.00   
                              4                            0.00    0.00   

LEVEL2                                 heart rate hematocrit hemoglobin  \
Aggregation Function                         mean       mean       mean   
subject_id hadm_id icustay_id hours_in                                    
3          145834  211552     0            143.00      27.03       8.72   
                              1            153.75       0.00       0.00   
                              2            137.75      30.00      10.10   
                              3 

In [235]:
print(f"Exclude {excluded_subject_ids.size} patients...")

patients = patients[~patients.index.get_level_values("subject_id").isin(excluded_subject_ids)]
print(patients.shape)

outcome = outcome[~outcome.index.get_level_values("subject_id").isin(excluded_subject_ids)]
print(outcome.shape)

vitals_labs_mean = vitals_labs_mean[~vitals_labs_mean.index.get_level_values("subject_id").isin(excluded_subject_ids)]
print(vitals_labs_mean.shape)

indicators = indicators[~indicators.index.get_level_values("subject_id").isin(excluded_subject_ids)]
print(indicators.shape)

Exclude 8406 patients...
(29137, 8)
(29137, 1)
(2368063, 27)
(2368063, 27)
